In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

real_file = '../App/Данные/OULU 01.01.2022 - 31.12.2024 форматированные.csv'
generated_file = '../App/data/generated_data.csv'

real = pd.read_csv(real_file)
generated = pd.read_csv(generated_file)

real_series = real.values.flatten()
generated_series = generated.values.flatten()

real_series = real_series[~np.isnan(real_series)]
generated_series = generated_series[~np.isnan(generated_series)]

def print_stats(name, series):
    print(f"\n Статистика для {name}:")
    print(f"Среднее значение:      {np.mean(series):.2f}")
    print(f"Медиана:               {np.median(series):.2f}")
    print(f"Минимум:               {np.min(series):.2f}")
    print(f"Максимум:              {np.max(series):.2f}")
    print(f"Стандартное отклонение:{np.std(series):.2f}")

print_stats("Сгенерированных данных", generated_series)
print_stats("Реальных данных", real_series)

def create_features(series, window=3):
    X, y = [], []
    for i in range(window, len(series) - 1):
        lag_features = series[i-window:i]                 
        ma = np.mean(lag_features)                 
        diff = series[i] - series[i-1]                 
        feature_vector = list(lag_features) + [ma, diff]
        X.append(feature_vector)
        y.append(series[i + 1])                      
    return np.array(X), np.array(y)

window_size = 10

X_gen, y_gen = create_features(generated_series, window=window_size)
X_real, y_real = create_features(real_series, window=window_size)

model_from_generated = LinearRegression()
model_from_generated.fit(X_gen, y_gen)
y_pred_from_generated = model_from_generated.predict(X_real)

model_from_real = LinearRegression()
model_from_real.fit(X_real, y_real)
y_pred_from_real = model_from_real.predict(X_real)

def print_metrics(title, y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"\n{title}")
    print(f" MSE  = {mse:.2f}")
    print(f" MAE  = {mae:.2f}")
    print(f" R²   = {r2:.3f}")

print_metrics("Предсказание реальных данных с модели, обученной на сгенерированных", y_real, y_pred_from_generated)
print_metrics("Предсказание реальных данных с модели, обученной на реальных", y_real, y_pred_from_real)



 Статистика для Сгенерированных данных:
Среднее значение:      6054.35
Медиана:               6053.98
Минимум:               5991.51
Максимум:              6117.98
Стандартное отклонение:15.52

 Статистика для Реальных данных:
Среднее значение:      6090.29
Медиана:               6055.00
Минимум:               4155.00
Максимум:              7213.00
Стандартное отклонение:323.16

Предсказание реальных данных с модели, обученной на сгенерированных
 MSE  = 16254.49
 MAE  = 101.59
 R²   = 0.844

Предсказание реальных данных с модели, обученной на реальных
 MSE  = 16075.44
 MAE  = 101.04
 R²   = 0.846


In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

real = pd.read_csv('../App/Данные/OULU 01.01.2022 - 31.12.2024 форматированные.csv')
generated = pd.read_csv('../App/data/generated_data.csv')

real_series = real.values.flatten()
generated_series = generated.values.flatten()

real_series = real_series[~np.isnan(real_series)]
generated_series = generated_series[~np.isnan(generated_series)]

# === Подготовка обучающих и тестовых данных ===
def prepare_xy(series):
    X = []
    y = []
    for i in range(len(series) - 1):
        X.append([series[i]])
        y.append(series[i + 1])
    return np.array(X), np.array(y)

X_gen, y_gen = prepare_xy(generated_series)  
X_real, y_real = prepare_xy(real_series)    

model_generated = LinearRegression()
model_generated.fit(X_gen, y_gen)
y_pred_from_generated = model_generated.predict(X_real)

model_real = LinearRegression()
model_real.fit(X_real, y_real)
y_pred_from_real = model_real.predict(X_real)

def print_metrics(title, y_true, y_pred):
    print(f"\n{title}")
    print(f"MSE:  {mean_squared_error(y_true, y_pred):.2f}")
    print(f"MAE:  {mean_absolute_error(y_true, y_pred):.2f}")
    print(f"R^2:  {r2_score(y_true, y_pred):.3f}")

print_metrics("Модель обучена на сгенерированных, тест на реальных", y_real, y_pred_from_generated)
print_metrics("Модель обучена и протестирована на реальных", y_real, y_pred_from_real)



Модель обучена на сгенерированных, тест на реальных
MSE:  34638.71
MAE:  148.79
R^2:  0.668

Модель обучена и протестирована на реальных
MSE:  27392.35
MAE:  131.98
R^2:  0.738
